In [ ]:
PATH ="C:\\Users\\GANESH\\Desktop\\pyspark\\dataset2\\"
FILE = "marketing_sample_for_careerbuilder_usa-careerbuilder_job_listing__20200401_20200630__30k_data.ldjson"

#### Dendencies

In [ ]:
import pyspark
from pyspark.sql import SparkSession as session
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas
pandas.set_option("display.max.column",None)
import re

#### Create Spark Session

In [ ]:
spark = session.builder.appName("job").getOrCreate()
spark_df=spark.read.json(PATH+FILE,primitivesAsString = 'true')
print("Spark Session Created Successfully")

In [ ]:
spark_df.printSchema()

#### Drop duplicates and Unused Columns

In [ ]:
spark_df =spark_df.drop('inferred_city','inferred_iso3_lang_code','inferred_salary_time_unit',\
                        'inferred_iso2_lang_code','inferred_state','last_expiry_check_date',\
                        'postdate_yyyymmdd')

In [ ]:
def dropnul(spark_df):
    null = spark_df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in spark_df.columns]).\
          collect()[0].asDict()
    col_to_drop = [key for key, value in null.items() if value > 0.7]
    spark_df = spark_df.drop(*col_to_drop)
    return spark_df
dropnul(spark_df)
new_spark_df = spark_df.transform(dropnul)
new_spark_df.show(10)

calculate number of jobs posted on daily basis, per each city

In [ ]:
jobs=spark_df.groupBy(["city"]).agg(
    count("post_date").alias("total_jobs_post"))
jobs = jobs.sort(desc("total_jobs_post")) 

# Convert to Pandas DataFrame and Save to Local
josbs_pd_df =jobs.toPandas()
josbs_pd_df.to_csv(PATH+'job_posts_data.csv',index=False)

In [ ]:
print("Number of jobs Posts by cities on Daily Basis")
jobs.show(10)

calculate average salary per job title and state

In [ ]:
sal =new_spark_df.select( 'job_title','state',split("salary_offered","[$/-]").getItem(1).cast('double').alias('salary'))
sal=sal.na.drop()
salary_df =sal.groupBy('job_title','state').agg(avg('salary').alias('salary in $/hour'),count('*').alias('count'))

In [ ]:
# Convert to Pandas DataFrame and Save to Local
avg_salary = salary_df.na.drop().toPandas()
avg_salary.to_csv(PATH+"avg_salary.csv", index=False)

In [ ]:
print("Average salary on the basis of Job title and State")
salary_df.show(10)

Identify the top 10 most active companies by number of positions opened

In [ ]:
company=spark_df.groupBy(["company_name"]).agg(count("post_date").alias("number_of_position"))
company =company.sort(desc("number_of_position")).limit(10)

# Convert to Pandas DataFrame and Save to Local
company_pd_df=company.toPandas()
company_pd_df.to_csv(PATH+'Top_10_companies.csv',index=False)

In [ ]:
print("Top 10 Most Active Companies")
company.show()

Create a UDF Function

In [ ]:
#create a UDF function to clean job description from HTML code contained inside
# analyze and comment on the performance of the UDF
# function. How the performance could be improved
import time
start_time = time.time()

clean_html =udf(lambda row: re.sub(re.compile('<.*?>'), '',row))
new_spark_df =new_spark_df.withColumn('html_job_description', clean_html("html_job_description"))
print(f"Execution time: {time.time() - start_time}")

# Using split() function cleanup HTML
df2 = new_spark_df.select(split(col("html_job_description"),"Job Description ",2)\
                          .getItem(1).alias("new_job_description")) \
                           .drop("html_job_description")

In [ ]:
spark.stop()
print("SparkSession stopped")